In [1]:
import torch
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import cv2 as cv
import tensorflow as tf

In [2]:
torch.cuda.is_available()

True

## **Feature Extractor**

In [3]:
model = tf.keras.applications.ConvNeXtTiny()

In [4]:
model.summary()

Model: "convnext_tiny"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_pres… │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stem  │ (None, 56, 56,    │      4,896 │ convnext_tiny_pr… │
│ (Sequential)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │      4,800 │ convnext_tiny_st… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     36,960 │ convnext_tiny_st… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │         96 │ convnext_tiny_st… │
│ (LayerScale)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│                     │ 96)               │            │ convnext_tiny_st… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │      4,800 │ add[0][0]         │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     36,960 │ convnext_tiny_st… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │         96 │ convnext_tiny_st

 Total params: 28,589,128 (109.06 MB)

 Trainable params: 28,589,128 (109.06 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
FE = tf.keras.Model(inputs=model.inputs, 
                        outputs=model.layers[-2].output)

In [6]:
FE.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_pres… │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stem  │ (None, 56, 56,    │      4,896 │ convnext_tiny_pr… │
│ (Sequential)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │      4,800 │ convnext_tiny_st… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     36,960 │ convnext_tiny_st… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │         96 │ convnext_tiny_st… │
│ (LayerScale)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│                     │ 96)               │            │ convnext_tiny_st… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │      4,800 │ add[0][0]         │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │     36,960 │ convnext_tiny_st… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 56, 56,    │         96 │ convnext_tiny_st

 Total params: 27,820,128 (106.13 MB)

 Trainable params: 27,820,128 (106.13 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
del model

## **Image Scale**

In [8]:
def cvt224(data):
    data224 = []
    for i in range(len(data)):
        x = cv.resize(data[i],(224,224))
        data224.append(x)
    return np.array(data224)
    

## **Data Augmentation**

In [9]:
keys = ['data', 'targets']

In [10]:
cwd = os.getcwd()
L_L = ['part_one_dataset','part_two_dataset']
S_P = os.path.join(cwd, 'Augmented_Data')

In [11]:
if not os.path.exists(S_P):
    os.mkdir(S_P)
    
for LF_N in L_L:
    LF = os.path.join(cwd,LF_N)
    SF = os.path.join(S_P,LF_N)
    LF_L = os.listdir(LF)
    if not os.path.exists(SF):
        os.mkdir(SF)
    for LFF_N in LF_L:
        SFF_N = LFF_N
        LFF = os.path.join(LF,LFF_N)
        if not os.path.isdir(LFF):
            continue
        SFF = os.path.join(SF,SFF_N)
        if not os.path.exists(SFF):
            os.mkdir(SFF)
        if not os.path.isdir(LFF):
            continue
        LFF_L = os.listdir(LFF)
        for LFFD_N in LFF_L:
            SFFD_N = LFFD_N.split('.')[0]
            LFFD = os.path.join(LFF,LFFD_N)
            SFFD = os.path.join(SFF,SFFD_N)
            print(LFFD)
            print(SFFD)
            data_set = torch.load(LFFD)
            data224  = cvt224(data_set[keys[0]])
            data_aug = FE.predict(data224)
            data_dict = {}
            data_dict[keys[0]] = data_aug
            try:
                y = data_set[keys[1]]
                data_dict[keys[1]] = y
            except KeyError:
                pass
                
            np.savez(SFFD,**data_dict)

/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/part_one_dataset/eval_data/10_eval_data.tar.pth
/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/Augmented_Data/part_one_dataset/eval_data/10_eval_data


/tmp/ipykernel_8165/1253482179.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_set = torch.load(LFFD)
I0000 00:00:1732433323.764259    8265 service.cc:145] XLA ser

 4/79 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step

I0000 00:00:1732433328.606555    8265 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


78/79 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

I0000 00:00:1732433334.816749    8450 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot', 128 bytes spill stores, 128 bytes spill loads

I0000 00:00:1732433335.164237    8442 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_12', 164 bytes spill stores, 164 bytes spill loads



79/79 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step
/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/part_one_dataset/eval_data/1_eval_data.tar.pth
/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/Augmented_Data/part_one_dataset/eval_data/1_eval_data
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step
/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/part_one_dataset/eval_data/2_eval_data.tar.pth
/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/Augmented_Data/part_one_dataset/eval_data/2_eval_data
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step
/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/part_one_dataset/eval_data/3_eval_data.tar.pth
/media/bulu/New Volume/.Bulu/.IIT Kanpur/1st Sem/ML/Mini Project 2/dataset/dataset/Augmented_Data/part_one_dataset/eval_data/3_eval_data
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step
/media/bulu/New Volume/.Bulu/.IIT Kan

In [16]:
x = np.load('Augmented_Data/part_one_dataset/train_data/1_train_data.npz')

In [18]:
x.keys()

KeysView(NpzFile 'Augmented_Data/part_one_dataset/train_data/1_train_data.npz' with keys: data, targets)

In [20]:
x['targets']

array([6, 9, 9, ..., 9, 9, 7])

In [26]:
x['data']

array([[ 0.05088265,  0.09383455, -0.30428627, ..., -0.0366103 ,
        -0.12699893, -0.27069393],
       [ 0.10114037, -0.2443133 ,  0.23039512, ...,  0.00342967,
        -0.22693317, -0.25912374],
       [ 0.30610308,  0.6338686 ,  0.03055203, ...,  0.24418205,
        -0.21335462,  0.21881278],
       ...,
       [-0.06314275, -0.20554787,  0.05189992, ...,  0.23893107,
        -0.11559833, -0.27119452],
       [ 0.23702395,  0.13334028, -0.64303154, ...,  0.3409128 ,
         0.02049099, -0.29506147],
       [ 0.10174888, -0.08598523,  0.05971277, ..., -0.10869779,
         0.13242738,  0.17165314]], dtype=float32)

In [28]:
x = np.load('Augmented_Data/part_two_dataset/train_data/1_train_data.npz')

In [30]:
x.keys()

KeysView(NpzFile 'Augmented_Data/part_two_dataset/train_data/1_train_data.npz' with keys: data)

In [32]:
x['targets']

KeyError: 'targets is not a file in the archive'

In [34]:
x['data']

array([[-0.05699619,  0.14959636, -0.10645515, ...,  0.46931782,
        -0.05003287, -0.11141731],
       [ 0.32238722,  0.07939089,  0.1493358 , ..., -0.04473737,
        -0.14457025,  0.68953604],
       [ 0.18489605, -0.14186466, -0.17745566, ...,  0.20418893,
        -0.20070021,  0.22715096],
       ...,
       [ 0.08793352, -0.08347078, -0.00953299, ..., -0.09196393,
        -0.24712098, -0.15964112],
       [-0.06853495, -0.13445155,  0.3322005 , ...,  0.7122564 ,
        -0.01229241, -0.31695113],
       [ 0.00701117,  0.247093  ,  0.10002419, ...,  0.24650142,
        -0.36291218,  0.06087071]], dtype=float32)